In [1]:
import numpy as np
import glob
import re
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#pandas
import pandas as pd

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

pyLDAvis.enable_notebook()

/home/sifr/uni/nlp/nlp_project/venv/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [2]:
df = pd.read_csv("data/ExtractedTweets.csv")
df.head()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...


### Extract tweet information

In [3]:
def find_retweeted(tweet):
    '''This function will extract the twitter handles of retweed people'''
    return re.findall('(?<=RT\\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', tweet)

def find_mentioned(tweet):
    '''This function will extract the twitter handles of people mentioned in the tweet'''
    return re.findall('(?<!RT\\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', tweet)

def find_hashtags(tweet):
    '''This function will extract hashtags'''
    return re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', tweet)

# make new columns for retweeted usernames, mentioned usernames and hashtags
df['retweeted'] = df['Tweet'].apply(find_retweeted)
df['mentioned'] = df['Tweet'].apply(find_mentioned)
df['hashtags'] = df['Tweet'].apply(find_hashtags)
df.head(30)

,Party,Handle,Tweet,retweeted,mentioned,hashtags
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",[],[],"[#SaveTheInternet, #NetNeutrality]"
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,[@WinterHavenSun],[@RepDarrenSoto],[]
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,[@NBCLatino],[@RepDarrenSoto],[]
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[@NALCABPolicy],"[@RepDarrenSoto, @LatinoLeader]",[#NALCABPolicy2018]
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,[@Vegalteno],"[@Pwr4PuertoRico, @RepDarrenSoto, @EspaillatNY]",[]
5,Democrat,RepDarrenSoto,RT @EmgageActionFL: Thank you to all who came ...,[@EmgageActionFL],[],[]
6,Democrat,RepDarrenSoto,Hurricane Maria left approx $90 billion in dam...,[],[],[]
7,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSo...,[@Tharryry],[@RepDarrenSoto],[#NetNeutrality]
8,Democrat,RepDarrenSoto,RT @HispanicCaucus: Trump's anti-immigrant pol...,[@HispanicCaucus],[],[]
9,Democrat,RepDarrenSoto,RT @RepStephMurphy: Great joining @WeAreUnidos...,[@RepStephMurphy],"[@WeAreUnidosUS, @RepDarrenSoto]",[#Orlando]


### Clean metioned, retweets and hashtags from tweet

In [4]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.append("rt")

def clean_retweeted(tweet):
    '''This function will extract the twitter handles of retweed people'''
    return re.sub('(?<=RT\\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)

def clean_mentioned(tweet):
    '''This function will extract the twitter handles of people mentioned in the tweet'''
    return re.sub('(?<!RT\\s)(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)

def clean_hashtags(tweet):
    '''This function will extract hashtags'''
    return re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)

def clean(tweet):
    # Remove mentions, retweets and hashtags
    tweet = clean_hashtags(tweet)
    tweet = clean_mentioned(tweet)
    tweet = clean_retweeted(tweet)
    tweet = tweet.lower()
    # Remove punctuation
    tweet = re.sub("[\\.,;:!/\\?]*", "", tweet)
    # Remove stop words
    tweet = ' '.join([word for word in tweet.split(' ') if not word in stop_words])
    # Remove multiple spaces
    tweet = re.sub("\\s+", " ", tweet)
    return tweet

df['TweetCleaned'] = df['Tweet'].apply(clean)
df.head()

,Party,Handle,Tweet,retweeted,mentioned,hashtags,TweetCleaned
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",[],[],"[#SaveTheInternet, #NetNeutrality]",today senate dems vote proud support similar l...
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,[@WinterHavenSun],[@RepDarrenSoto],[],winter resident alta vista teacher one severa...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,[@NBCLatino],[@RepDarrenSoto],[],noted hurricane maria left approximately $90 ...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[@NALCABPolicy],"[@RepDarrenSoto, @LatinoLeader]",[#NALCABPolicy2018],meeting thanks taking time meet ed marucci gu...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,[@Vegalteno],"[@Pwr4PuertoRico, @RepDarrenSoto, @EspaillatNY]",[],hurricane season starts june 1st puerto rico’...


### Lemmatize the text

In [5]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def lemmatization(tweet, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    doc = nlp(tweet)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    final = " ".join(new_text)
    return final

df['TweetCleaned'] = df['TweetCleaned'].apply(lemmatization)
df.head()

,Party,Handle,Tweet,retweeted,mentioned,hashtags,TweetCleaned
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",[],[],"[#SaveTheInternet, #NetNeutrality]",today dem vote proud support similar legislati...
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,[@WinterHavenSun],[@RepDarrenSoto],[],winter resident teacher several recognize nati...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,[@NBCLatino],[@RepDarrenSoto],[],note hurricane maria leave approximately damag...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[@NALCABPolicy],"[@RepDarrenSoto, @LatinoLeader]",[#NALCABPolicy2018],meeting thank take time
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,[@Vegalteno],"[@Pwr4PuertoRico, @RepDarrenSoto, @EspaillatNY]",[],hurricane season start readinesswell ♂ 😡


In [6]:
gensim.utils.simple_preprocess("today vote proud support similar legislation", deacc=True)
df.head()

,Party,Handle,Tweet,retweeted,mentioned,hashtags,TweetCleaned
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",[],[],"[#SaveTheInternet, #NetNeutrality]",today dem vote proud support similar legislati...
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,[@WinterHavenSun],[@RepDarrenSoto],[],winter resident teacher several recognize nati...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,[@NBCLatino],[@RepDarrenSoto],[],note hurricane maria leave approximately damag...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[@NALCABPolicy],"[@RepDarrenSoto, @LatinoLeader]",[#NALCABPolicy2018],meeting thank take time
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,[@Vegalteno],"[@Pwr4PuertoRico, @RepDarrenSoto, @EspaillatNY]",[],hurricane season start readinesswell ♂ 😡


In [7]:
def gen_words(tweet):
    # Use gensims simple simple_preprocess to remove accents, lowercase and tokenize
    return gensim.utils.simple_preprocess(tweet, deacc=True)

df['TweetWords'] = df['TweetCleaned'].apply(gen_words)
df.head()

,Party,Handle,Tweet,retweeted,mentioned,hashtags,TweetCleaned,TweetWords
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",[],[],"[#SaveTheInternet, #NetNeutrality]",today dem vote proud support similar legislati...,"[today, dem, vote, proud, support, similar, le..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,[@WinterHavenSun],[@RepDarrenSoto],[],winter resident teacher several recognize nati...,"[winter, resident, teacher, several, recognize..."
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,[@NBCLatino],[@RepDarrenSoto],[],note hurricane maria leave approximately damag...,"[note, hurricane, maria, leave, approximately,..."
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[@NALCABPolicy],"[@RepDarrenSoto, @LatinoLeader]",[#NALCABPolicy2018],meeting thank take time,"[meeting, thank, take, time]"
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,[@Vegalteno],"[@Pwr4PuertoRico, @RepDarrenSoto, @EspaillatNY]",[],hurricane season start readinesswell ♂ 😡,"[hurricane, season, start, readinesswell]"


In [8]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(df['TweetWords'], min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[df['TweetWords']], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(tweetWords):
    return bigram[tweetWords]

def make_trigrams(bigrams):
    return trigram[bigram[bigrams]]

df['bigramms'] =  df['TweetWords'].apply(make_bigrams)
df['trigrams'] = df['bigramms'].apply(make_trigrams)
df.head(30)

,Party,Handle,Tweet,retweeted,mentioned,hashtags,TweetCleaned,TweetWords,bigramms,trigrams
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",[],[],"[#SaveTheInternet, #NetNeutrality]",today dem vote proud support similar legislati...,"[today, dem, vote, proud, support, similar, le...","[today, dem, vote, proud, support, similar, le...","[today, dem, vote, proud, support, similar, le..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,[@WinterHavenSun],[@RepDarrenSoto],[],winter resident teacher several recognize nati...,"[winter, resident, teacher, several, recognize...","[winter, resident, teacher, several, recognize...","[winter, resident, teacher, several, recognize..."
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,[@NBCLatino],[@RepDarrenSoto],[],note hurricane maria leave approximately damag...,"[note, hurricane, maria, leave, approximately,...","[note, hurricane, maria, leave, approximately,...","[note, hurricane, maria, leave, approximately,..."
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,[@NALCABPolicy],"[@RepDarrenSoto, @LatinoLeader]",[#NALCABPolicy2018],meeting thank take time,"[meeting, thank, take, time]","[meeting, thank, take, time]","[meeting, thank, take, time]"
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,[@Vegalteno],"[@Pwr4PuertoRico, @RepDarrenSoto, @EspaillatNY]",[],hurricane season start readinesswell ♂ 😡,"[hurricane, season, start, readinesswell]","[hurricane_season, start, readinesswell]","[hurricane_season, start, readinesswell]"
5,Democrat,RepDarrenSoto,RT @EmgageActionFL: Thank you to all who came ...,[@EmgageActionFL],[],[],thank come orlando gala successful night possible,"[thank, come, orlando, gala, successful, night...","[thank, come, orlando, gala, successful, night...","[thank, come, orlando, gala, successful, night..."
6,Democrat,RepDarrenSoto,Hurricane Maria left approx $90 billion in dam...,[],[],[],leave damage yet allocate rebuild grid,"[leave, damage, yet, allocate, rebuild, grid]","[leave, damage, yet, allocate, rebuild, grid]","[leave, damage, yet, allocate, rebuild, grid]"
7,Democrat,RepDarrenSoto,RT @Tharryry: I am delighted that @RepDarrenSo...,[@Tharryry],[@RepDarrenSoto],[#NetNeutrality],delight voting rule find,"[delight, voting, rule, find]","[delight, voting, rule, find]","[delight, voting, rule, find]"
8,Democrat,RepDarrenSoto,RT @HispanicCaucus: Trump's anti-immigrant pol...,[@HispanicCaucus],[],[],trump anti - immigrant policy hurt small busin...,"[trump, anti, immigrant, policy, hurt, small, ...","[trump, anti, immigrant, policy, hurt, small_b...","[trump, anti_immigrant, policy, hurt, small_bu..."
9,Democrat,RepDarrenSoto,RT @RepStephMurphy: Great joining @WeAreUnidos...,[@RepStephMurphy],"[@WeAreUnidosUS, @RepDarrenSoto]",[#Orlando],great join roundtable federal issue affect cen...,"[great, join, roundtable, federal, issue, affe...","[great, join, roundtable, federal, issue, affe...","[great, join, roundtable, federal, issue, affe..."


### Remove tokens which occur very often, since they do not contain semantic meaning and tokens which occur only a few times to reduce data noise

In [9]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(df['trigrams'])

texts = df['trigrams']

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

corpus[:5]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1)],
 [(10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1)],
 [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)],
 [(23, 1), (24, 1), (25, 1), (26, 1)],
 [(27, 1), (28, 1), (29, 1)]]

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha="auto")


KeyboardInterrupt: 

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

Evaluating the model see: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#5preparestopwords

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha="auto")

        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()